In [1]:
import sys
sys.path.append('..')

In [2]:
import math
import re
from collections import namedtuple
from pathlib import Path

import numpy as np
from PIL import Image, ImageDraw, ImageOps

from allometry.const import DATA_DIR

# import skimage
# from skimage import io, filters, feature
# from skimage import transform as xform
# from skimage import morphology as morph
# from skimage import segmentation as seg

In [3]:
SHEET = DATA_DIR / 'allometry_sheets'
# SHEET = SHEET / 'Biomass_Fish_Families_FamilyProgram'
SHEET = SHEET / 'Amphibians_Reptiles_RingBinder'
SHEET = SHEET / '00010.tif'

In [4]:
MIN_PIXELS = 20
PADDING = 1
THETA = 90.0
PIXEL_THRESHOLD = 230
SCALE = 0.5

In [5]:
Score = namedtuple('Score', 'score where')
Where = namedtuple('Where', 'row type')

### Get the image

In [6]:
image = Image.open(SHEET).convert('L')
# image = image.rotate(THETA, expand=True, fillcolor='white')
# image = ImageOps.scale(image, SCALE)
# display(image)

### Binarize the image

In [7]:
binary = image.point(lambda x: 255 if x < PIXEL_THRESHOLD else 0)
# display(binary)

### Profile projection

In [8]:
def profile_projection(binary, threshold=20, axis=1):
    """Characters in the image via a profile projection."""
    data = np.array(binary).copy() / 255

    proj = data.sum(axis=axis)
    proj = proj > threshold
    proj = proj.astype(int)

    prev = np.insert(proj[:-1], 0, 0)
    curr = np.insert(proj[1:], 0, 0)
    where = np.where(curr != prev)[0]
    where = where.tolist()

    splits = np.array_split(proj, where)
    score = sum(len(s) for s in splits if s[0] == 0)

    where = where if where[0] == 0 else ([0] + where)
    where = [Where(w, s[0]) for w, s in zip(where, splits)]

    return Score(score, where)

In [9]:
Box = namedtuple('Box', 'left right')


def merge_boxes(lefts, rights, inside=4, outside=30):
    boxes = [Box(lefts[0], rights[0])]

    for next_left, next_right in zip(lefts[1:], rights[1:]):
        left, right = boxes[-1].left, boxes[-1].right
        print(left, right, next_left, next_right)

        if (next_left - right) <= inside and (next_right - left) <= outside:
            boxes.pop()
            boxes.append(Box(left, next_right))
        else:
            boxes.append(Box(next_left, next_right))

    print()

    return boxes

### Find rows

In [10]:
score = profile_projection(binary, threshold=20)

In [11]:
top_lines = [w.row - PADDING for w in score.where if w.type == 1]
bottom_lines = [w.row + PADDING for w in score.where if w.type == 0][1:]

### Find characters in rows

In [12]:
width, height = image.size

marked = binary.convert('RGB')
draw = ImageDraw.Draw(marked)

In [13]:
for top, bottom in zip(top_lines, bottom_lines):
    # if bottom - top < MIN_PIXELS:
    #     continue

    row = binary.crop((0, top, width, bottom))
    score = profile_projection(row, axis=0, threshold=0)

    row_width, row_height = row.size

    # draw.rectangle((1, top, row_width - 1, bottom), outline=(255, 0, 0))

    left_lines = [w.row - PADDING for w in score.where if w.type == 1]
    right_lines = [w.row + PADDING for w in score.where if w.type == 0][1:]

    boxes = merge_boxes(left_lines, right_lines)
    
    for box in boxes:
        # if right - left > 10:
        draw.rectangle((box.left, top, box.right, bottom), outline=(0, 255, 255))

2030 2057 2057 2692
2057 2692 2791 2977
2791 2977 2984 3210


395 421 458 473
458 473 473 482
458 482 483 488
458 488 528 555
528 555 594 600
594 600 601 614
594 614 613 623
594 623 662 689
662 689 728 757
728 757 796 813
796 813 813 823
796 823 863 889
863 889 929 956
929 956 997 1023
997 1023 1063 1073
1063 1073 1072 1090
1063 1090 1128 1155
1128 1155 1194 1214
1194 1214 1213 1224
1194 1224 1262 1277
1262 1277 1276 1290
1262 1290 1330 1359
1330 1359 1398 1402
1398 1402 1402 1406
1398 1406 1406 1423
1398 1423 1465 1483
1465 1483 1482 1492
1465 1492 1530 1558
1530 1558 1598 1604
1598 1604 1603 1612
1598 1612 1611 1624
1598 1624 1666 1695

567 591 600 628
600 628 663 695
663 695 700 729
700 729 742 753
742 753 1300 1327
1300 1327 1334 1362
1334 1362 1367 1377
1367 1377 1376 1395
1367 1395 1407 1419
1407 1419 1432 1463
1432 1463 1468 1494
1468 1494 2402 2429
2402 2429 2435 2465
2435 2465 2470 2498
2470 2498 2503 2529
2503 2529 2535 2566
2535 2566 2569 2577
2569 2577 2576 2582
2569 2582 2

In [14]:
path = DATA_DIR / 'marked.jpg'
marked.save(path, 'JPEG')

### Experiments to see if we can improve the image quality

In [15]:
# canny = np.asarray(binary)

# canny = feature.canny(canny, sigma=3, low_threshold=0.5, high_threshold=0.75)
# canny = skimage.img_as_ubyte(canny)

# canny = Image.fromarray(canny)

# path = DATA_DIR / 'canny.jpg'
# canny.save(path, 'JPEG')

# display(canny)

In [16]:
# selem = morph.disk(3)

# closed = np.asarray(binary)

# closed = morph.binary_closing(closed, selem)
# closed = skimage.img_as_ubyte(closed)

# closed = Image.fromarray(closed)

# path = DATA_DIR / 'closed.jpg'
# closed.save(path, 'JPEG')

# display(closed)

In [17]:
# skel = np.asarray(binary) / 255

# skel = morph.skeletonize(skel, method='lee')
# skel = skimage.img_as_ubyte(skel)

# skel = Image.fromarray(skel)

# path = DATA_DIR / 'skel.jpg'
# skel.save(path, 'JPEG')

# display(skel)

In [18]:
# thin = np.asarray(binary)

# thin = morph.thin(thin)
# thin = skimage.img_as_ubyte(thin)

# thin = Image.fromarray(thin)

# path = DATA_DIR / 'thin.jpg'
# thin.save(path, 'JPEG')

# display(thin)

### Deskew image

In [19]:
# NOTE: Pillow's rotate is only by whole numbers. Maybe an affine?

# for angle in np.arange(0.0, 2.1, 0.1):
#     rotated = binary.rotate(angle, fillcolor='black', expand=True)
#     best = profile_projection(rotated, angle, best)

# for angle in np.arange(178.0, 180.0, 0.1):
#     rotated = binary.rotate(angle, fillcolor='black', expand=True)
#     best = profile_projection(rotated, angle, best)